# Sample Environment

In [1]:
import gymnasium  as gym 
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

from stable_baselines3.common.env_checker import check_env

In [2]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]), dtype=np.float32)
        # Set start temp
        self._obs = 38 + random.randint(-3,3)
        # Set shower length
        self._shower_length = 60
        # Set placeholder for info
        self._info = {}
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self._obs += action -1 
        # Reduce shower length by 1 second
        self._shower_length -= 1 
        
        # Calculate reward
        if self._obs >=37 and self._obs <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self._shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        
        # Return step information
        return self._obs, reward, done, False, self._info


    def render(self):
        # Implement viz
        pass
    
    def reset(self, seed=None):
        super().reset(seed=seed)

        # Reset shower temperature
        self._obs = np.array([38 + random.randint(-3,3)]).astype(np.float32)
        # Reset shower time
        self._shower_length = 60 
        return self._obs, self._info
    
env=ShowerEnv()
check_env(env, warn=True)

In [3]:
env.reset()

(array([37.], dtype=float32), {'TimeLimit.truncated': False})

In [4]:
env.observation_space.sample()

array([10.939985], dtype=float32)

In [5]:
env.action_space.sample()

np.int64(1)

# Test Environment

In [7]:
episodes = 5
for episode in range(1, episodes+1):
    _obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, _, _ = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-42
Episode:2 Score:-16
Episode:3 Score:20
Episode:4 Score:-60
Episode:5 Score:-6


# Train Model

In [10]:
# log_path = os.path.join('Training', 'Logs')
# model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -45.6    |
| time/              |          |
|    fps             | 1329     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


# Evaluate & Export

In [17]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(np.float64(-57.8), np.float64(2.4413111231467406))

In [18]:
model.save('PPO')